In [1]:
import os
import shutil
import random
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import cv2

In [2]:
def download_dataset(kaggle_username, kaggle_key, dataset_name):
    !pip install -q kaggle
    from kaggle.api.kaggle_api_extended import KaggleApi
    
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files(dataset_name, path='./', unzip=True)
    dataset_dir = './' + dataset_name.split('/')[1]
    os.makedirs(dataset_dir, exist_ok=True)
    files = os.listdir('./')
    for file in files:
        if file.endswith('.csv') or file.endswith('.jpg') or file.endswith('.png'):
            shutil.move(file, dataset_dir)

#https://www.kaggle.com/datasets/alessiocorrado99/animals10

In [3]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (64, 64))
    image = image / 255.0
    pca = PCA(n_components=32)
    image_flattened = image.reshape(-1, 3)
    image_transformed = pca.fit_transform(image_flattened)
    return image_transformed

In [4]:
def load_and_split_data(data_dir, test_size=0.2, val_size=0.2):
    file_names = os.listdir(data_dir)
    images = []
    labels = []
    for file_name in file_names:
        image_path = os.path.join(data_dir, file_name)
        image = preprocess_image(image_path)
        images.append(image)
        labels.append(get_label_from_file_name(file_name))    
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)   
    return X_train, X_val, X_test, y_train, y_val, y_test

In [5]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(32,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [6]:
def train_model(X_train, y_train, X_val, y_val, model):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(np.array(X_train), np.array(y_train), validation_data=(np.array(X_val), np.array(y_val)), epochs=10)
    return model, history



In [7]:
def evaluate_model(model, X_train, y_train, X_val, y_val):
    train_loss, train_acc = model.evaluate(np.array(X_train), np.array(y_train), verbose=0)
    val_loss, val_acc = model.evaluate(np.array(X_val), np.array(y_val), verbose=0)
    print('Train Accuracy:', train_acc)
    print('Validation Accuracy:', val_acc)

    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()



In [8]:
def scrape_data(url):
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    # Aquí debes escribir el código para encontrar los elementos HTML relevantes y extraer los datos que necesitas.
    # Puedes utilizar los métodos y selectores de BeautifulSoup para navegar por el árbol HTML y seleccionar los elementos deseados.

    data = []
    items = soup.select('.item')  # Selector para los elementos con clase 'item'
    for item in items:
        title = item.select_one('.title').text  # Selector para el título del elemento
        description = item.select_one('.description').text  # Selector para la descripción del elemento
        data.append({'title': title, 'description': description})

    return data

In [9]:
def detect_animal(image_path, model):
    preprocessed_image = preprocess_image(image_path)
    input_image = np.expand_dims(preprocessed_image, axis=0)
    predictions = model.predict(input_image)
    predicted_class = np.argmax(predictions[0])
    if predicted_class == 1:
        print("Animal detectado, activar bocinas")
    else:
        print("No se detectó un animal")

In [10]:
from google.colab import drive
drive.mount('/content/drive')  # Montar Google Drive

Mounted at /content/drive


In [11]:
from google.colab import drive
import os

data_dir = '/content/drive/MyDrive/Colab Notebooks/proyecto_sis_int/raw-img'  # Ruta a la carpeta del conjunto de datos en Google Drive

def load_images_from_subfolders(data_dir):
    images = []
    labels = []
    subfolders = os.listdir(data_dir)
    for subfolder in subfolders:
        subfolder_path = os.path.join(data_dir, subfolder)
        if os.path.isdir(subfolder_path):
            files = os.listdir(subfolder_path)
            for file in files:
                image_path = os.path.join(subfolder_path, file)
                image = preprocess_image(image_path)
                images.append(image)
                labels.append(subfolder)  # Usar el nombre de la subcarpeta como etiqueta/clase

    return images, labels

images, labels = load_images_from_subfolders(data_dir)
X_train, X_val, X_test, y_train, y_val, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


ValueError: ignored

In [ ]:
model = create_model()
model, history = train_model(X_train, y_train, X_val, y_val, model)

In [ ]:
evaluate_model(model, X_train, y_train, X_val, y_val)

In [ ]:
url = 'https://www.example.com'
data = scrape_data(url)

In [ ]:
image_path = 'ruta/a/una/imagen.jpg'
detect_animal(image_path, model)